In [1]:
!if not exist "./files" mkdir files
# Download Face detection XML 
!curl -L -o ./files/haarcascade_frontalface_default.xml https://raw.githubusercontent.com/opencv/opencv/master/data/haarcascades/haarcascade_frontalface_default.xml
# Download emotion trained data
!curl -L -o ./files/emotion_model.hdf5 https://mechasolution.vn/source/blog/AI-tutorial/Emotion_Recognition/emotion_model.hdf5

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0  908k    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  908k  100  908k    0     0  1019k      0 --:--:-- --:--:-- --:--:-- 1020k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:--  0:00:01 --:--:--     0
  0     0    0     0    0     0      0      0 --:--:--  0:00:02 --:--:--     0
 14  878k   14  128k    0     0  40477      0  0:00:22  0:00:03  0:00:19 40477
100  878k  100  878k    0     0   248k      0  0:0

In [3]:
import cv2
import numpy as np   
from keras.preprocessing.image import img_to_array
from keras.models import load_model

# Face detection XML load and trained model loading
face_detection = cv2.CascadeClassifier('files/haarcascade_frontalface_default.xml')
emotion_classifier = load_model('files/emotion_model.hdf5', compile=False)
EMOTIONS = ["Angry" ,"Disgusting","Fearful", "Happy", "Sad", "Surpring", "Neutral"]

# Video capture using webcam
camera = cv2.VideoCapture(0)

while True:
    # Capture image from camera
    ret, frame = camera.read()
    
    # 좌우반전
    frame = cv2.flip(frame, 1)
    
    # Convert color to gray scale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # Face detection in frame
    faces = face_detection.detectMultiScale(gray,
                                            scaleFactor=1.1,
                                            minNeighbors=5,
                                            minSize=(30,30))
    
    # Create empty image
    canvas = np.zeros((250, 300, 3), dtype="uint8")
    
    # Perform emotion recognition only when face is detected
    if len(faces) > 0:
        # For the largest image
        face = sorted(faces, reverse=True, key=lambda x: (x[2] - x[0]) * (x[3] - x[1]))[0]
        (fX, fY, fW, fH) = face
        # Resize the image to 48x48 for neural network
        roi = gray[fY:fY + fH, fX:fX + fW]
        roi = cv2.resize(roi, (48, 48))
        roi = roi.astype("float") / 255.0
        roi = img_to_array(roi)
        roi = np.expand_dims(roi, axis=0)
        
        # Emotion predict(모든 감정)
        preds = emotion_classifier.predict(roi)[0]
        # 가장 높은 감정의 score
        emotion_probability = np.max(preds)

        # 가장 높은 감정 
        label = EMOTIONS[preds.argmax()]
        
        # Assign labeling
        cv2.putText(frame, label, (fX, fY - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.45, (0, 0, 255), 2)
        cv2.rectangle(frame, (fX, fY), (fX + fW, fY + fH), (0, 0, 255), 2)
 
        # Label printing
        for (i, (emotion, prob)) in enumerate(zip(EMOTIONS, preds)):
            text = "{}: {:.2f}%".format(emotion, prob * 100)    
            w = int(prob * 300)
            cv2.rectangle(canvas, (7, (i * 35) + 5), (w, (i * 35) + 35), (0, 0, 255), -1)
            cv2.putText(canvas, text, (10, (i * 35) + 23), cv2.FONT_HERSHEY_SIMPLEX, 0.45, (255, 255, 255), 2)

    # Open two windows
    ## Display image ("Emotion Recognition")
    ## Display probabilities of emotion
    cv2.imshow('Emotion Recognition', frame)
    #cv2.imshow("Probabilities", canvas)
    
    # q to quit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Clear program and close windows
camera.release()
cv2.destroyAllWindows()

[0.26444063 0.01086899 0.07350546 0.00766815 0.07869078 0.00672223
 0.55810374]
[0.32297042 0.01345858 0.04425587 0.00372983 0.05128149 0.00324916
 0.56105465]
[0.25813502 0.01000685 0.05839767 0.00601658 0.03601467 0.00806286
 0.6233663 ]
[0.23704936 0.00493056 0.02706346 0.00689821 0.04308938 0.0083814
 0.67258763]
[0.22722425 0.01606781 0.05601081 0.00574623 0.0438811  0.00499196
 0.6460778 ]
[0.2980202  0.01527605 0.08558681 0.00910092 0.07090664 0.00905774
 0.5120516 ]
[0.21310948 0.03959491 0.1013763  0.01279614 0.05456212 0.01730894
 0.5612521 ]
[0.39988565 0.03650904 0.11761793 0.01173858 0.06482778 0.0075564
 0.36186454]
[0.29185227 0.00749743 0.11227977 0.00970323 0.07681772 0.00639109
 0.4954585 ]
[0.24476317 0.01022203 0.0849827  0.00952865 0.05463634 0.01160837
 0.5842588 ]
[0.30781266 0.01754542 0.09797046 0.00625517 0.08513573 0.00718427
 0.4780963 ]
[0.41495594 0.03162232 0.10137963 0.00592234 0.06573661 0.0056128
 0.3747704 ]
[0.3185356  0.02414705 0.07111936 0.0124205